# European Parliament Voted Resolutions

In [ ]:
! pip install fuzzywuzzy

In [4]:
! pip install python-docx 

In [6]:
! pip install xmltodict

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import fuzz
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import requests
from docx import Document

import sys
sys.path.append("..")
from src.data_extraction import *
from src.dataset import DatasetGenerator
from src.get_votes import ta2votes
from pathlib import Path

pd.set_option('max_colwidth', 200)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
input_folder = '../data/raw/'
output_folder = '../data/processed/'

_ = DatasetGenerator(input_folder, output_folder)

In [5]:
df_filtered = _.filter_df()
df_filtered = df_filtered.drop(['document_type', 'document_language', 'document_pdf', 'document_corrigenda_pdf', 'document_corrigenda_doc'], axis=1)
df_filtered

,document_identifier,document_title,document_parliamentary_term,document_date,document_public_register_notation,document_doc,document_subject_matter,document_directory_code,document_adopts,document_procedure,document_ep_number,document_URI
13,TA-10-2024-0036,EU actions against the Russian shadow fleets and ensuring a full enforcement of sanctions against Russia,10,2024-11-14,P10_TA(2024)0036,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0036_en.docx,http://publications.europa.eu/resource/authority/subject-matter/PESC,http://publications.europa.eu/resource/authority/dir-eu-legal-act/18,https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0162; https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0166; https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0163; https://data.europa...,https://data.europarl.europa.eu/eli/dl/proc/2024-2885,PE764.162,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0036
21,TA-10-2024-0021,Establishing the Ukraine Loan Cooperation Mechanism and providing exceptional macro-financial assistance to Ukraine,10,2024-10-22,P10_TA(2024)0021,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0021_en.docx,http://publications.europa.eu/resource/authority/subject-matter/ASSI; http://publications.europa.eu/resource/authority/subject-matter/EXT,http://publications.europa.eu/resource/authority/dir-eu-legal-act/115020,https://data.europarl.europa.eu/eli/dl/doc/A-10-2024-0006,https://data.europarl.europa.eu/eli/dl/proc/2024-0234,PE765.025,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0021
64,TA-10-2024-0016,Strengthening Moldova's resilience against Russian interference ahead of the upcoming presidential elections and a constitutional referendum on EU integration,10,2024-10-09,P10_TA(2024)0016,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0016_en.docx,http://publications.europa.eu/resource/authority/subject-matter/ADH; http://publications.europa.eu/resource/authority/subject-matter/EXT,http://publications.europa.eu/resource/authority/dir-eu-legal-act/11401030,https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0081; https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0085; https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0080; https://data.europa...,https://data.europarl.europa.eu/eli/dl/proc/2024-2821,PE764.090,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0016
66,TA-10-2024-0012,Continued financial and military support to Ukraine by EU Member States,10,2024-09-19,P10_TA(2024)0012,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0012_en.docx,http://publications.europa.eu/resource/authority/subject-matter/PESC,http://publications.europa.eu/resource/authority/dir-eu-legal-act/18,https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0028; https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0031; https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0033; https://data.europa...,https://data.europarl.europa.eu/eli/dl/proc/2024-2799,PE762.612,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0012
69,TA-10-2024-0003,The need for the EU's continuous support for Ukraine,10,2024-07-17,P10_TA(2024)0003,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0003_en.docx,http://publications.europa.eu/resource/authority/subject-matter/PESC,http://publications.europa.eu/resource/authority/dir-eu-legal-act/18,https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0007,https://data.europarl.europa.eu/eli/dl/proc/2024-2721,PE762.593,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0003
88,TA-9-2024-0051,"Humanitarian situation in Gaza, the need to reach a ceasefire and the risks of regional escalation",9,2024-01-18,P9_TA(2024)0051,https://data.europarl.europa.eu/distribution/doc/TA-9-2024-0051_en.docx,http://publications.europa.eu/resource/authority/subject-matter/PESC; http://publications.europa.eu/resource/authority/subject-matter/HUMA,http://publications.europa.eu/resource/authority/dir-eu-legal-act/18,https://data.europarl.europa.eu/eli/dl/doc/B

In [6]:
len(df_filtered)

33

In [7]:
sample = df_filtered.iloc[19]
sample

document_identifier                                                                                                                                                                                                           TA-9-2023-0029
document_title                                                                                                                                                                                          Preparation of the EU-Ukraine Summit
document_parliamentary_term                                                                                                                                                                                                                9
document_date                                                                                                                                                                                                                     2023-02-02
document_public_register_notation                   

In [8]:
ta2votes(date=sample['document_date'], k=sample['document_parliamentary_term'], document_adopts=sample['document_adopts'])

{'For': '489',
 'Against': '36',
 'Abstention': '49',
 'PoliticalGroup': {'ECR': {'For': 51, 'Abstention': 3},
  'ID': {'For': 22, 'Against': 13, 'Abstention': 18},
  'NI': {'For': 6, 'Against': 9, 'Abstention': 8},
  'PPE': {'For': 139, 'Abstention': 3},
  'Renew': {'For': 86, 'Against': 3, 'Abstention': 3},
  'S&D': {'For': 114, 'Against': 3, 'Abstention': 7},
  'The Left': {'For': 15, 'Against': 10, 'Abstention': 6},
  'Verts/ALE': {'For': 56, 'Abstention': 5}}}

In [98]:
# Define the directory paths
processed_dir = Path("../data/processed")
cleaned_dir = Path("../data/cleaned")

# Create the cleaned directory if it doesn't exist
cleaned_dir.mkdir(parents=True, exist_ok=True)

In [99]:
def clean_text(text):
    """
    Cleans and normalizes the input text.
    - Removes the 'TEXTS ADOPTED' header and its associated line.
    - Removes non-essential metadata like resolution codes.
    - Normalizes spaces and line breaks.
    """
    # Explicitly remove the "TEXTS ADOPTED" line
    text = re.sub(r"(?i)^TEXTS ADOPTED\s*\n", "", text)
    # Remove metadata patterns (example: P9_TA(2023)0051)
    text = re.sub(r"P\d+_TA\(\d{4}\)\d{4}", "", text)
    # Remove codes like PE741.462
    text = re.sub(r"PE\d+\.\d+", "", text)
    # Remove any residual leading/trailing spaces
    text = text.strip()
    # Replace multiple line breaks with a single one
    text = re.sub(r"\n\s*\n", "\n", text)
    # Normalize spaces
    text = re.sub(r"[ \t]+", " ", text)
    return text

In [100]:
def process_text_files(input_dir, output_dir):
    """
    Cleans and normalizes all text files in the input directory and saves them in the output directory.
    """
    for file_path in input_dir.glob("*.txt"):
        # Read the text content
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read()
        
        # Clean the text
        cleaned_text = clean_text(text)
        
        # Save the cleaned text
        output_path = output_dir / file_path.name
        with open(output_path, "w", encoding="utf-8") as file:
            file.write(cleaned_text)
        print(f"Processed and saved: {output_path}")

In [102]:
# Process all files in the 'processed' directory
process_text_files(processed_dir, cleaned_dir)

Processed and saved: ../data/cleaned/document_67.txt
Processed and saved: ../data/cleaned/document_98.txt
Processed and saved: ../data/cleaned/document_799.txt
Processed and saved: ../data/cleaned/document_348.txt
Processed and saved: ../data/cleaned/document_604.txt
Processed and saved: ../data/cleaned/document_65.txt
Processed and saved: ../data/cleaned/document_70.txt
Processed and saved: ../data/cleaned/document_417.txt
Processed and saved: ../data/cleaned/document_761.txt
Processed and saved: ../data/cleaned/document_211.txt
Processed and saved: ../data/cleaned/document_402.txt
Processed and saved: ../data/cleaned/document_158.txt
Processed and saved: ../data/cleaned/document_880.txt
Processed and saved: ../data/cleaned/document_117.txt
Processed and saved: ../data/cleaned/document_471.txt
Processed and saved: ../data/cleaned/document_673.txt
Processed and saved: ../data/cleaned/document_12.txt
Processed and saved: ../data/cleaned/document_460.txt
Processed and saved: ../data/clea